<a href="https://colab.research.google.com/github/TryDovick/MosiNLP/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключаем библиотеки

In [9]:
import numpy as np
from sklearn.preprocessing import StandardScaler

Функции активации


In [2]:
#ReLU (Rectified Linear Unit)
#Возвращает 0 для отрицательных входов и сам вход для положительных
def relu(x):
    return np.maximum(0, x)


In [3]:
#Производная ReLU
#Возвращает 1 для положительных входов и 0 для отрицательных
def relu_derivative(x):
    return (x > 0).astype(float)

In [4]:
def softmax(x):
    exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

In [11]:
class GuitarClassifier:


    def __init__(self, input_size=5, hidden_size=6, output_size=3):
        # Инициализация весов (метод He)
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2./input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2./hidden_size)

        self.b2 = np.zeros((1, output_size))

    def forward(self, X):

        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.probs = softmax(self.z2)
        return self.probs

    def backward(self, X, y, learning_rate=0.01):
        m = X.shape[0]  # количество примеров

        # Градиенты выходного слоя
        delta3 = self.probs
        delta3[range(m), y] -= 1
        delta3 /= m


        # Градиенты скрытого слоя
        dW2 = np.dot(self.a1.T, delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = np.dot(delta3, self.W2.T) * relu_derivative(self.z1)

        # Градиенты входного слоя
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # Обновление весов
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

    def train(self, X, y, epochs=5000, batch_size=16):
        for epoch in range(epochs):
            # Мини-пакетный градиентный спуск
            indices = np.random.permutation(len(X))
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            for i in range(0, len(X), batch_size):
                X_batch = X_shuffled[i:i+batch_size]
                y_batch = y_shuffled[i:i+batch_size]

                self.forward(X_batch)
                self.backward(X_batch, y_batch)

            # Логирование каждые 500 эпох
            if epoch % 500 == 0:
                probs = self.forward(X)
                loss = -np.mean(np.log(probs[range(len(X)), y]))
                preds = np.argmax(probs, axis=1)
                accuracy = np.mean(preds == y)
                print(f"Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)



Созадем список данных

In [6]:
X = np.array([
    [6, 25.5, 43, 1.5, 0],  # Акустическая
    [6, 24.75, 42, 3.2, 1],  # Электрогитара
    [4, 34, 45, 4.1, 1],     # Бас-гитара
    [12, 25.5, 48, 2.1, 0],  # 12-струнная акустическая
    [6, 25.5, 43, 3.5, 1],   # Полуакустическая
    [5, 30, 44, 3.8, 1],     # 5-струнный бас
    [6, 24.75, 41, 3.1, 1],  # Электрогитара
    [4, 32, 46, 4.0, 1],     # Бас-гитара
    [6, 25.5, 43, 1.7, 0],   # Акустическая
    [7, 26.5, 44, 3.6, 1]    # 7-струнная электрогитара
])

y = np.array([0, 1, 2, 0, 1, 2, 1, 2, 0, 1])


In [10]:
# Нормализация данных
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# обучаем модель
model = GuitarClassifier(input_size=5, hidden_size=6, output_size=3)
model.train(X_normalized, y, epochs=3000)

# Тут я создал примеры, какие типы гитар должны быть выведены
test_guitars = np.array([
    [6, 25.0, 42, 1.6, 0],  #Акустическая
    [6, 24.5, 42, 3.3, 1],  #Электрогитара
    [5, 32, 45, 4.2, 1]     #5-струнный бас
])

test_guitars_normalized = scaler.transform(test_guitars)
predictions = model.predict(test_guitars_normalized)

# Выводим результаты
guitar_types = ['Акустическая', 'Электрогитара', 'Бас-гитара']
for i, guitar in enumerate(test_guitars):
    print(f"Гитара {i+1}: {guitar} => {guitar_types[predictions[i]]}")

Epoch 0: Loss = 1.6228, Accuracy = 0.0000
Epoch 500: Loss = 0.2768, Accuracy = 1.0000
Epoch 1000: Loss = 0.1389, Accuracy = 1.0000
Epoch 1500: Loss = 0.0832, Accuracy = 1.0000
Epoch 2000: Loss = 0.0548, Accuracy = 1.0000
Epoch 2500: Loss = 0.0382, Accuracy = 1.0000
Гитара 1: [ 6.  25.  42.   1.6  0. ] => Акустическая
Гитара 2: [ 6.  24.5 42.   3.3  1. ] => Электрогитара
Гитара 3: [ 5.  32.  45.   4.2  1. ] => Бас-гитара
